In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import requests
import json

s=requests.Session()
s.headers = {
    'authority': 'homes.hdb.gov.sg',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'accept': 'application/json, text/plain, */*',
    'sessionid': 'a',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36',
    'content-type': 'application/json',
    'origin': 'https://homes.hdb.gov.sg',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://homes.hdb.gov.sg/home/finding-a-flat',
    'accept-language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7',
}


In [3]:

data = '{"location":"","coordinates":[["",""]],"range":"2","priceRangeLower":"0","priceRangeUpper":"0","flatType":"","waitingTime":"","modeOfSale":"BTO,SBF","floorRange":"","remainingLeaseRangeLower":1,"remainingLeaseRangeUpper":99,"ethnicGroup":"Chinese","rank":"Location, Price Range, Flat Type, Remaining Lease"}'

response = s.post('https://homes.hdb.gov.sg/home-api/public/v1/map/getCoordinatesByFilters', data=data)

response


<Response [200]>

In [4]:
jsonstr=json.loads(response.text)
detailedUrl='https://resources.homes.hdb.gov.sg/nf/{}.xml'
# https://resources.homes.hdb.gov.sg/nf/2021-05/bto/tg_d2c4/2021-05_BTO_VEdfRDJDNF8xNjE5NTAyODQxMDYz.xml
#all project details xml link follow this format - works well for BTO
for x in jsonstr:
    storageDict={}
#     to try and reduce API calls on price range
    x['properties']['description']=x['properties']['description'][0]
    if x['properties']['listingType']=='BTO':
        projPath=x['properties']['description'].get('photoUrl','').rpartition('/')[0]
        newUrl=detailedUrl.format(projPath+'/'+x['properties']['description']['projectId'])
    elif x['properties']['listingType']=='SBF':
        ballotQtr=x['properties']['description']['ballotQtr']
        projectID=x['properties']['description']['projectId']
        projDict='{"projectId":"'+projectID+'"}'
        storageDict={}
        x['availability']=json.loads(s.post('https://homes.hdb.gov.sg/home-api/public/v1/launch/getProjectAvailabilityAndEthnic',data=projDict).text)
        for block,value in x['availability']['projectBlockFlatTypeInfoMap'].items():
            for flatType in value.keys():
                if storageDict.get(flatType) is None:
                    priceDict='{"projectId":"'+projectID+'","flatType":"'+flatType+'"}'
                    storageDict[flatType]=s.post('https://homes.hdb.gov.sg/home-api/public/v1/launch/getPriceRangeAcrossTownByProjectIdAndFlatType',data=priceDict).text
                x['availability']['projectBlockFlatTypeInfoMap'][block][flatType]['areaPrice']=storageDict[flatType]
                    
        newUrl=detailedUrl.format(ballotQtr+'/sbf/unit_xml/UNIT_'+projectID)
    details=s.get(newUrl)
    x['details']=details.text

In [5]:
df=pd.json_normalize(jsonstr,max_level=1)
df.head(5)

,coordinates,details,properties.address,properties.listingType,properties.description,properties.hdbCategory,properties.region,availability.availabilitySet,availability.projectBlockFlatTypeInfoMap
0,"[1.2730969, 103.8082856]","<?xml version=""1.0"" encoding=""UTF-8"" standalon...",Telok Blangah Beacon,BTO,{'photoUrl': '2021-05/bto/bm_n1c39b/bm_n1c39b_...,1,CENTRAL REGION,NaN,NaN
1,"[1.3271563, 103.8892293]","<?xml version=""1.0"" encoding=""UTF-8"" standalon...",MacPherson Weave,BTO,{'photoUrl': '2021-05/bto/gl_n1c13/gl_n1c13_th...,1,CENTRAL REGION,NaN,NaN
2,"[1.4317643, 103.7830138]","<?xml version=""1.0"" encoding=""UTF-8"" standalon...",Woodgrove Ascent,BTO,{'photoUrl': '2021-05/bto/wl_n3c16/wl_n3c16_th...,1,NORTH REGION,NaN,NaN
3,"[1.3607471, 103.7243003]","<?xml version=""1.0"" encoding=""UTF-8"" standalon...",Garden Bloom @ Tengah,BTO,{'photoUrl': '2021-05/bto/tg_d2c4/tg_d2c4_thum...,1,WEST REGION,NaN,NaN
4,"[1.3766745369726854, 103.73789561376547]","<?xml version=""1.0"" encoding=""UTF-8"" standalon...",Sunshine Court,SBF,"{'maxRemainingLease': '45', 'stage': 'Launch',...",2,WEST REGION,"[4312, 4313, 4314, 4315, 4316, 4317]","{'476A': {'2-room flexi': {'block': '476A', 'f..."


In [6]:
df['details'][5]

'<?xml version="1.0" encoding="UTF-8" standalone="no"?>\n<template>\n    <standard-project-info>\n        <unit-details>\n            <availability>\n                <block>\n                    <block-number>488B</block-number>\n                    <flat-type>\n                        <type>2-Room Flexi</type>\n                        <level>\n                            <level-number>07</level-number>\n                            <unit>\n                                <unit-id>4318</unit-id>\n                                <unit-number>141</unit-number>\n                                <area>47</area>\n                                <price>84900</price>\n                                <deliveryPossessionDate>Keys Available</deliveryPossessionDate>\n                                <estimatedCompletionDate>-</estimatedCompletionDate>\n                                <short-lease-price>\n                                    <tenure-45>84900</tenure-45>\n                              

In [7]:
df.to_hdf('hdbSaleDATA.h5','HDBSALES')
df.to_pickle('hdbSalesDATA.pkl')
with open("hdbSaleDATA.json", "w") as write_file:
    json.dump(jsonstr, write_file)

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:2621: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['coordinates', 'details', 'properties.address',
       'properties.listingType', 'properties.description',
       'properties.hdbCategory', 'properties.region',
       'availability.availabilitySet',
       'availability.projectBlockFlatTypeInfoMap'],
      dtype='object')]

  encoding=encoding,


In [8]:
jsonstr[6]['availability']

{'availabilitySet': ['4320', '4321', '4322', '4323'],
 'projectBlockFlatTypeInfoMap': {'574': {'2-room flexi': {'block': '574',
    'flatType': '2-room flexi',
    'chinese': 1,
    'malay': 1,
    'indian': 1,
    'areaPrice': '{"priceRangeUpper":132300,"priceRangeLower":39100}'}},
  '570': {'2-room flexi': {'block': '570',
    'flatType': '2-room flexi',
    'chinese': 1,
    'malay': 1,
    'indian': 1,
    'areaPrice': '{"priceRangeUpper":132300,"priceRangeLower":39100}'}},
  '571': {'2-room flexi': {'block': '571',
    'flatType': '2-room flexi',
    'chinese': 2,
    'malay': 2,
    'indian': 2,
    'areaPrice': '{"priceRangeUpper":132300,"priceRangeLower":39100}'}}}}